<a href="https://colab.research.google.com/github/alaaosman801/30-Day-Hacker-Rack-Challenge/blob/main/feature%20extraction%20with%20data%20agumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats
! mkdir train
! unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip




import os, shutil, pathlib
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")




def make_subset(subset_name, startidx, endidx):
  for category in("cat", "dog"):
    dir = new_base_dir/subset_name/category
    os.makedirs(dir)
    fnames = [f"{category}.{i}.jpg" for i in range(startidx, endidx)]
    for fname in fnames:
      shutil.copyfile(src=original_dir/fname,dst= dir/ fname)

make_subset("train", startidx=0, endidx=1000)
make_subset("validation", startidx=1000, endidx=1500)
make_subset("test", startidx=1500, endidx=2500)





from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
401 - Unauthorized
mkdir: cannot create directory ‘train’: File exists
unzip:  cannot find or open dogs-vs-cats.zip, dogs-vs-cats.zip.zip or dogs-vs-cats.zip.ZIP.
unzip:  cannot find or open train.zip, train.zip.zip or train.zip.ZIP.


FileExistsError: ignored

In [ ]:
from tensorflow import keras
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
conv_base.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
conv_base.trainable = True
print("This is the number of trainable weights "
      "before freezing the conv base:", len(conv_base.trainable_weights))

This is the number of trainable weights before freezing the conv base: 26


In [ ]:
conv_base.trainable = False
print("This is the number of trainable weights "
      "after freezing the conv base:", len(conv_base.trainable_weights))

This is the number of trainable weights after freezing the conv base: 0


In [ ]:
from keras import layers
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=25,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/50
63/63 [==============================] - 2533s 40s/step - loss: 21092.3418 - accuracy: 0.5090 - val_loss: 0.6863 - val_accuracy: 0.5460
Epoch 2/50
63/63 [==============================] - 2501s 40s/step - loss: 0.7886 - accuracy: 0.5100 - val_loss: 0.6932 - val_accuracy: 0.5010
Epoch 3/50
63/63 [==============================] - 2480s 39s/step - loss: 0.6937 - accuracy: 0.4835 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/50
21/63 [=========>....................] - ETA: 24:29 - loss: 0.6934 - accuracy: 0.5074

KeyboardInterrupt: ignored